# LabSheet 6 
## 17000475

## Constituency Parsing

In [1]:
sentence = 'the quick brown fox jumped over the lazy dog'

In [2]:
import os
java_path = 'C:/Program Files/Java/jdk-13.0.1'
os.environ['JAVAHOME'] = java_path
os.environ['CLASSPATH'] = 'F:/Downloads/stanford-parser-4.2.0/stanford-parser-full-2020-11-17'

from nltk.parse.stanford import StanfordParser

scp = StanfordParser(model_path='edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz')


C:\Users\Kavishka\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: The StanfordParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPParser instead.
  


In [4]:
result = list(scp.raw_parse(sentence))
print(result[0])

(ROOT
  (S
    (NP (DT the) (JJ quick) (JJ brown) (NN fox))
    (VP (VBD jumped) (PP (IN over) (NP (DT the) (JJ lazy) (NN dog))))))


In [5]:
result[0].draw()

![image](/Screenshot 2021-07-01 185522.png)

In [9]:
import nltk
from nltk.grammar import Nonterminal
from nltk.corpus import treebank

In [10]:
training_set = treebank.parsed_sents()

print(training_set[1])

(S
  (NP-SBJ (NNP Mr.) (NNP Vinken))
  (VP
    (VBZ is)
    (NP-PRD
      (NP (NN chairman))
      (PP
        (IN of)
        (NP
          (NP (NNP Elsevier) (NNP N.V.))
          (, ,)
          (NP (DT the) (NNP Dutch) (VBG publishing) (NN group))))))
  (. .))


In [11]:
# extract the productions for all annotated training sentences
treebank_productions = list(
                        set(production 
                            for sent in training_set  
                            for production in sent.productions()
                        )
                    )

treebank_productions[0:10]

[NN -> 'are',
 NNP -> 'Si',
 NNS -> 'bombs',
 CC -> 'or',
 VBZ -> 'rates',
 NN -> 'atmosphere',
 NNS -> 'write-downs',
 NNS -> 'Ringers',
 VBG -> 'soliciting',
 VP -> VBD NP PP-BNF]

In [14]:
# add productions for each word, POS tag
for word, tag in treebank.tagged_words():
	t = nltk.Tree.fromstring("("+ tag + " " + word  +")")
	for production in t.productions():
		treebank_productions.append(production)

# build the PCFG based grammar  
treebank_grammar = nltk.grammar.induce_pcfg(Nonterminal('S'), 
                                         treebank_productions)

# build the parser
viterbi_parser = nltk.ViterbiParser(treebank_grammar)

# get sample sentence tokens
tokens = nltk.word_tokenize(sentence)

# get parse tree for sample sentence
#result = list(viterbi_parser.parse(tokens))

In [16]:
# get tokens and their POS tags from pattern package
#from pattern.en import tag as pos_tagger
#tagged_sent = pos_tagger(sentence)

# use NLTK POS tagger instead
tokens = nltk.word_tokenize(sentence)
tagged_sent = nltk.pos_tag(tokens)

print(tagged_sent)

[('the', 'DT'), ('quick', 'JJ'), ('brown', 'NN'), ('fox', 'NN'), ('jumped', 'VBD'), ('over', 'IN'), ('the', 'DT'), ('lazy', 'JJ'), ('dog', 'NN')]


In [17]:
# extend productions for sample sentence tokens
for word, tag in tagged_sent:
    t = nltk.Tree.fromstring("("+ tag + " " + word  +")")
    for production in t.productions():
        treebank_productions.append(production)

# rebuild grammar
treebank_grammar = nltk.grammar.induce_pcfg(Nonterminal('S'), 
                                         treebank_productions)                                         

# rebuild parser
viterbi_parser = nltk.ViterbiParser(treebank_grammar)

# get parse tree for sample sentence
result = list(viterbi_parser.parse(tokens))

print(result[0])
result[0].draw()

(S
  (NP-SBJ-126 (DT the) (JJ quick) (NN brown) (NN fox))
  (VP
    (VBD jumped)
    (ADVP-DIR (RP over))
    (NP-TMP (DT the) (JJ lazy) (NN dog)))) (p=2.61338e-35)


![image2](Screenshot 2021-07-01 221702.png)

## Dependency Parsing

In [6]:
sentence = 'Colorless green ideas sleep furiously'

from spacy.lang.en import English
parser = English()
parsed_sent = parser(sentence)

dependency_pattern = '{left}<---{word}[{w_type}]--->{right}\n--------'
for token in parsed_sent:
    print(dependency_pattern.format(word=token.orth_, 
                                  w_type=token.dep_,
                                  left=[t.orth_ 
                                            for t 
                                            in token.lefts],
                                  right=[t.orth_ 
                                             for t 
                                             in token.rights]))

[]<---Colorless[]--->[]
--------
[]<---green[]--->[]
--------
[]<---ideas[]--->[]
--------
[]<---sleep[]--->[]
--------
[]<---furiously[]--->[]
--------


In [7]:
java_path = 'C:/Program Files/Java/jdk-13.0.1'
os.environ['JAVAHOME'] = java_path
os.environ['CLASSPATH'] = 'F:/Downloads/stanford-parser-4.2.0/stanford-parser-full-2020-11-17'
from nltk.parse.stanford import StanfordDependencyParser
sdp = StanfordDependencyParser(model_path='edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz')

C:\Users\Kavishka\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.
  """


In [8]:
from graphviz import Source
result = list(sdp.raw_parse(sentence))
dep_tree = [parse for parse in result][0]
print(dep_tree)
dep_tree

defaultdict(<function DependencyGraph.__init__.<locals>.<lambda> at 0x000002047D312CA8>,
            {0: {'address': 0,
                 'ctag': 'TOP',
                 'deps': defaultdict(<class 'list'>, {'root': [4]}),
                 'feats': None,
                 'head': None,
                 'lemma': None,
                 'rel': None,
                 'tag': 'TOP',
                 'word': None},
             1: {'address': 1,
                 'ctag': 'JJ',
                 'deps': defaultdict(<class 'list'>, {}),
                 'feats': '_',
                 'head': 3,
                 'lemma': '_',
                 'rel': 'amod',
                 'tag': 'JJ',
                 'word': 'Colorless'},
             2: {'address': 2,
                 'ctag': 'JJ',
                 'deps': defaultdict(<class 'list'>, {}),
                 'feats': '_',
                 'head': 3,
                 'lemma': '_',
                 'rel': 'amod',
                 'tag': 'JJ',
        

Exception: Cannot find the dot binary from Graphviz package

In [9]:
dep_tree_dot_repr = [parse for parse in result][0].to_dot()
source = Source(dep_tree_dot_repr, filename="dep_tree", format="png")
#source.view()

In [19]:
import nltk
tokens = nltk.word_tokenize(sentence)

dependency_rules = """
'fox' -> 'The' | 'brown'
'quick' -> 'fox' | 'is' | 'and' | 'jumping'
'jumping' -> 'he' | 'is' | 'dog'
'dog' -> 'over' | 'the' | 'lazy'
"""

dependency_grammar = nltk.grammar.DependencyGrammar.fromstring(dependency_rules)
print(dependency_grammar)

Dependency grammar with 12 productions
  'fox' -> 'The'
  'fox' -> 'brown'
  'quick' -> 'fox'
  'quick' -> 'is'
  'quick' -> 'and'
  'quick' -> 'jumping'
  'jumping' -> 'he'
  'jumping' -> 'is'
  'jumping' -> 'dog'
  'dog' -> 'over'
  'dog' -> 'the'
  'dog' -> 'lazy'


In [28]:
tokens

['Colorless', 'green', 'ideas', 'sleep', 'furiously']

In [23]:
dp = nltk.ProjectiveDependencyParser(dependency_grammar)
res = [item for item in dp.parse(tokens)]
res
tree = res[1] 
print(tree)

[]

In [ ]:
tree.draw()   

## POS Tagging

In [10]:
sentence = "I saw the man with the telescope but he didn't see me"


# Using NLTK's built-in tagger based on PTB
import nltk
tokens = nltk.word_tokenize(sentence)
tagged_sent = nltk.pos_tag(tokens, tagset='universal')
print(tagged_sent)

[('I', 'PRON'), ('saw', 'VERB'), ('the', 'DET'), ('man', 'NOUN'), ('with', 'ADP'), ('the', 'DET'), ('telescope', 'NOUN'), ('but', 'CONJ'), ('he', 'PRON'), ('did', 'VERB'), ("n't", 'ADV'), ('see', 'VERB'), ('me', 'PRON')]


In [11]:
from nltk.corpus import treebank
data = treebank.tagged_sents()
train_data = data[:3500]
test_data = data[3500:]
print(train_data[0])

[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]


In [13]:
# SAQ 1: How much data is there for training, testing? 
# rain - 3500 and Test - 414
len(test_data)

414

In [15]:
# SAQ 2: What is the last training sentence; test sentence?
train_data[-1]

[('The', 'DT'),
 ('acquisition', 'NN'),
 ('of', 'IN'),
 ('87-store', 'JJ'),
 ('Weisfield', 'NNP'),
 ("'s", 'POS'),
 ('raises', 'VBZ'),
 ('Ratners', 'NNP'),
 ("'s", 'POS'),
 ('U.S.', 'NNP'),
 ('presence', 'NN'),
 ('to', 'TO'),
 ('450', 'CD'),
 ('stores', 'NNS'),
 ('.', '.')]

In [16]:
# SAQ 2: What is the last training sentence; test sentence?
test_data[-1]

[('Trinity', 'NNP'),
 ('said', 'VBD'),
 ('0', '-NONE-'),
 ('it', 'PRP'),
 ('plans', 'VBZ'),
 ('*-1', '-NONE-'),
 ('to', 'TO'),
 ('begin', 'VB'),
 ('delivery', 'NN'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('first', 'JJ'),
 ('quarter', 'NN'),
 ('of', 'IN'),
 ('next', 'JJ'),
 ('year', 'NN'),
 ('.', '.')]

In [18]:

# Default 'naive' tagger - tags all words with a given tag!
from nltk.tag import DefaultTagger
dt = DefaultTagger('NN') # Can specify any default tag - NN gives best score - why? yes. It has most of the data
# Test score and example sentence tag output
print(dt.evaluate(test_data))
print(dt.tag(tokens))

0.1454158195372253
[('I', 'NN'), ('saw', 'NN'), ('the', 'NN'), ('man', 'NN'), ('with', 'NN'), ('the', 'NN'), ('telescope', 'NN'), ('but', 'NN'), ('he', 'NN'), ('did', 'NN'), ("n't", 'NN'), ('see', 'NN'), ('me', 'NN')]


In [19]:
# Regex tagger
from nltk.tag import RegexpTagger
# Define 'fixed' regex tag patterns
patterns = [
        (r'.*ing$', 'VBG'),               # gerunds
        (r'.*ed$', 'VBD'),                # simple past
        (r'.*es$', 'VBZ'),                # 3rd singular present
        (r'.*ould$', 'MD'),               # modals
        (r'.*\'s$', 'NN$'),               # possessive nouns
        (r'.*s$', 'NNS'),                 # plural nouns
        (r'^-?[0-9]+(.[0-9]+)?$', 'CD'),  # cardinal numbers
        (r'.*', 'NN')                     # nouns (default) ... 
]
rt = RegexpTagger(patterns)

# Test score and example sentence tag output
print(rt.evaluate(test_data))
print(rt.tag(tokens))

0.24039113176493368
[('I', 'NN'), ('saw', 'NN'), ('the', 'NN'), ('man', 'NN'), ('with', 'NN'), ('the', 'NN'), ('telescope', 'NN'), ('but', 'NN'), ('he', 'NN'), ('did', 'NN'), ("n't", 'NN'), ('see', 'NN'), ('me', 'NN')]


In [20]:
# Training your own tagger
# 1. using n-gram taggers and combining them with backoff
# 2. using naive bayes (statistical) model
# 3. using maximum entropy (classifier) model

## N gram taggers
from nltk.tag import UnigramTagger # Context insentitive
from nltk.tag import BigramTagger  # Considers previous word
from nltk.tag import TrigramTagger # Considers previous 2 words

# Traing the taggers
ut = UnigramTagger(train_data)
bt = BigramTagger(train_data)
tt = TrigramTagger(train_data)

In [21]:
# Test UnigramTagger score and example sentence tag output
print(ut.evaluate(test_data))
print(ut.tag(tokens))

# Test BigramTagger score and example sentence tag output
print(bt.evaluate(test_data))
print(bt.tag(tokens))

# Test TrigramTagger score and example sentence tag output
print(tt.evaluate(test_data))
print(tt.tag(tokens))

0.8607803272340013
[('I', 'PRP'), ('saw', 'VBD'), ('the', 'DT'), ('man', 'NN'), ('with', 'IN'), ('the', 'DT'), ('telescope', None), ('but', 'CC'), ('he', 'PRP'), ('did', 'VBD'), ("n't", 'RB'), ('see', 'VB'), ('me', 'PRP')]
0.13466937748087907
[('I', 'PRP'), ('saw', None), ('the', None), ('man', None), ('with', None), ('the', None), ('telescope', None), ('but', None), ('he', None), ('did', None), ("n't", None), ('see', None), ('me', None)]
0.08064672281924679
[('I', 'PRP'), ('saw', None), ('the', None), ('man', None), ('with', None), ('the', None), ('telescope', None), ('but', None), ('he', None), ('did', None), ("n't", None), ('see', None), ('me', None)]


In [22]:
# Combining all 3 n-gram taggers with backoff (smoothing)
def combined_tagger(train_data, taggers, backoff=None):
    for tagger in taggers:
        backoff = tagger(train_data, backoff=backoff)
    return backoff

ct = combined_tagger(train_data=train_data, 
                     taggers=[UnigramTagger, BigramTagger, TrigramTagger],
                     backoff=rt)

# Test Combined n-gram tagger score and example sentence tag output
print(ct.evaluate(test_data))       
print(ct.tag(tokens))

0.9094781682641108
[('I', 'PRP'), ('saw', 'VBD'), ('the', 'DT'), ('man', 'NN'), ('with', 'IN'), ('the', 'DT'), ('telescope', 'NN'), ('but', 'CC'), ('he', 'PRP'), ('did', 'VBD'), ("n't", 'RB'), ('see', 'VB'), ('me', 'PRP')]


In [23]:
# Treating POS tagging as a classification problem
# We use the ClassifierBasedPOSTagger class to build a classifier by specifying some
# classification algorithm - here the NaiveBayes abd Maxent algorithms which are passed
# to the class via the classifier_builder parameter
from nltk.classify import NaiveBayesClassifier, MaxentClassifier
from nltk.tag.sequential import ClassifierBasedPOSTagger

# First a Naive Bayes (statistical) classifier
nbt = ClassifierBasedPOSTagger(train=train_data,
                               classifier_builder=NaiveBayesClassifier.train)

# Test NBC tagger score and example sentence tag output
print(nbt.evaluate(test_data))
print(nbt.tag(tokens))  

0.9306806079969019
[('I', 'PRP'), ('saw', 'VBD'), ('the', 'DT'), ('man', 'NN'), ('with', 'IN'), ('the', 'DT'), ('telescope', 'NN'), ('but', 'CC'), ('he', 'PRP'), ('did', 'VBD'), ("n't", 'RB'), ('see', 'VB'), ('me', 'PRP')]


In [24]:
# Finally a Maximum entropy classifier (that would take sometime)
# met = ClassifierBasedPOSTagger(train=train_data,
#                               classifier_builder=MaxentClassifier.train)

met = ClassifierBasedPOSTagger(train=train_data, 
                               classifier_builder=lambda train_feats: MaxentClassifier.train(train_feats, max_iter=10))

# Test Maxent tagger score and example sentence tag output
print(met.evaluate(test_data))                           
print(met.tag(tokens))


  ==> Training (10 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -3.82864        0.007
             2          -0.76176        0.957
C:\Users\Kavishka\anaconda3\lib\site-packages\nltk\classify\maxent.py:1392: RuntimeWarning: overflow encountered in power
  exp_nf_delta = 2 ** nf_delta
C:\Users\Kavishka\anaconda3\lib\site-packages\nltk\classify\maxent.py:1394: RuntimeWarning: invalid value encountered in multiply
  sum1 = numpy.sum(exp_nf_delta * A, axis=0)
C:\Users\Kavishka\anaconda3\lib\site-packages\nltk\classify\maxent.py:1395: RuntimeWarning: invalid value encountered in multiply
  sum2 = numpy.sum(nf_exp_nf_delta * A, axis=0)
         Final               nan        0.984
0.9270984606447865
[('I', 'PRP'), ('saw', 'VBD'), ('the', 'DT'), ('man', 'NN'), ('with', 'IN'), ('the', 'DT'), ('telescope', 'NN'), ('but', 'CC'), ('he', 'PRP'), ('did', 'VBD'), ("n't", 'RB'), ('see', 'VB'), ('me', 'PRP')]


In [25]:
# Final accuracies
print('Tagger accuracies:')
print()
print('Default tagger %.2f' %dt.evaluate(test_data))
print('Regex tagger %.2f' %rt.evaluate(test_data))
print('Unigram tagger %.2f' %ut.evaluate(test_data))
print('Bigram tagger %.2f' %bt.evaluate(test_data))
print('Trigram tagger %.2f' %tt.evaluate(test_data))
print('Combined tagger %.2f' %ct.evaluate(test_data))
print('Naive Bayes tagger %.2f' %nbt.evaluate(test_data))
print('Maxent tagger %.2f' %met.evaluate(test_data))


Tagger accuracies:

Default tagger 0.15
Regex tagger 0.24
Unigram tagger 0.86
Bigram tagger 0.13
Trigram tagger 0.08
Combined tagger 0.91
Naive Bayes tagger 0.93
Maxent tagger 0.93


## Shallow_parsing

In [26]:
# Training your own chunker using chunked treeband data - again made available in NLTK!
# As before (with tagging) we first divide the data into training and testing sets
from nltk.corpus import treebank_chunk
data = treebank_chunk.chunked_sents()
train_data = data[0:3500]
test_data = data[3500:]
print(train_data[0])

simple_sentence = 'the brown fox jumped over the lazy dog'


(S
  (NP Pierre/NNP Vinken/NNP)
  ,/,
  (NP 61/CD years/NNS)
  old/JJ
  ,/,
  will/MD
  join/VB
  (NP the/DT board/NN)
  as/IN
  (NP a/DT nonexecutive/JJ director/NN Nov./NNP 29/CD)
  ./.)


In [27]:
# Can use tagger from package pattern.en if using Python 2.x
# from pattern.en import tag
# tagged_sentence = tag(sentence)

import nltk
from nltk.chunk import RegexpParser
tokens = nltk.word_tokenize(simple_sentence)
tagged_simple_sent = nltk.pos_tag(tokens)
print(tagged_simple_sent)

[('the', 'DT'), ('brown', 'JJ'), ('fox', 'NN'), ('jumped', 'VBD'), ('over', 'IN'), ('the', 'DT'), ('lazy', 'JJ'), ('dog', 'NN')]


In [28]:
# We first define our grammars using regex pattern using the RegexpParser
# We can specify which patterns we want to segment in a sentence as *chunks*
chunk_grammar = """
NP: {<DT>?<JJ>*<NN.*>}
VP: {<VBD><IN>}
"""
rc = RegexpParser(chunk_grammar)
c = rc.parse(tagged_simple_sent)
print(c)

(S
  (NP the/DT brown/JJ fox/NN)
  (VP jumped/VBD over/IN)
  (NP the/DT lazy/JJ dog/NN))


In [29]:
# We sometimes want to specify which patterns we DO NOT want to segment in a sentence
# so that we can *chunk* all the others
chink_grammar = """
NP: {<JJ|NN>+} # chunk only adjective-noun pair as NP
"""

rc = RegexpParser(chink_grammar)
c = rc.parse(tagged_simple_sent)
print(c)


(S
  the/DT
  (NP brown/JJ fox/NN)
  jumped/VBD
  over/IN
  the/DT
  (NP lazy/JJ dog/NN))


In [30]:
# A more realistic grammar for chunking
grammar = """
NP: {<DT>?<JJ>?<NN.*>}  
ADJP: {<JJ>}
ADVP: {<RB.*>}
PP: {<IN>}      
VP: {<MD>?<VB.*>+}

"""

# And a more realistic sentence as input
sentence = 'the brown fox is quick and he may jump over the lazy dog'
tokens = nltk.word_tokenize(sentence)
tagged_sent = nltk.pos_tag(tokens)

rc = RegexpParser(grammar)
c = rc.parse(tagged_sent)
print(c)
print(rc.evaluate(test_data))
# The performance is not great!
# Why is this?

(S
  (NP the/DT brown/JJ fox/NN)
  (VP is/VBZ)
  (ADJP quick/JJ)
  and/CC
  he/PRP
  (VP may/MD jump/VB)
  (PP over/IN)
  (NP the/DT lazy/JJ dog/NN))
ChunkParse score:
    IOB Accuracy:  46.1%%
    Precision:     19.9%%
    Recall:        43.3%%
    F-Measure:     27.3%%


In [31]:
# We have acecss to a utility function tree2conlltags which extracts word, tag and
# chunk triples from annotated text
from nltk.chunk.util import tree2conlltags, conlltags2tree
# Let's take a slightly more typical sentence from our data
train_sent = train_data[7]
print(train_sent)



(S
  (NP A/DT Lorillard/NNP spokewoman/NN)
  said/VBD
  ,/,
  ``/``
  (NP This/DT)
  is/VBZ
  (NP an/DT old/JJ story/NN)
  ./.)


In [32]:
# We extract the POS and chunk tags using tree2conlltags function which returns a list of tuples
wtc = tree2conlltags(train_sent)
wtc

[('A', 'DT', 'B-NP'),
 ('Lorillard', 'NNP', 'I-NP'),
 ('spokewoman', 'NN', 'I-NP'),
 ('said', 'VBD', 'O'),
 (',', ',', 'O'),
 ('``', '``', 'O'),
 ('This', 'DT', 'B-NP'),
 ('is', 'VBZ', 'O'),
 ('an', 'DT', 'B-NP'),
 ('old', 'JJ', 'I-NP'),
 ('story', 'NN', 'I-NP'),
 ('.', '.', 'O')]

In [33]:
# We can 'reverse' this to output a shallow tree using the conlltags2tree function
tree = conlltags2tree(wtc)
print(tree)

(S
  (NP A/DT Lorillard/NNP spokewoman/NN)
  said/VBD
  ,/,
  ``/``
  (NP This/DT)
  is/VBZ
  (NP an/DT old/JJ story/NN)
  ./.)


In [34]:
# We can use these features to train a 'combined' chunker as we did for POS tagging
def conll_tag_chunks(chunk_sents):
  tagged_sents = [tree2conlltags(tree) for tree in chunk_sents]
  return [[(t, c) for (w, t, c) in sent] for sent in tagged_sents]
  
def combined_tagger(train_data, taggers, backoff=None):
    for tagger in taggers:
        backoff = tagger(train_data, backoff=backoff)
    return backoff

In [35]:
from nltk.tag import UnigramTagger, BigramTagger
from nltk.chunk import ChunkParserI

# We create a new class to use the word, POS and Chunk tag features to train a chunker
# that is able to 'backoff' from bigram to a unigram model as before
# Can you have another layer for trigram and back off to this model?
class NGramTagChunker(ChunkParserI):
    
  def __init__(self, train_sentences, 
               tagger_classes=[UnigramTagger, BigramTagger]):
    train_sent_tags = conll_tag_chunks(train_sentences)
    self.chunk_tagger = combined_tagger(train_sent_tags, tagger_classes)

  def parse(self, tagged_sentence):
    if not tagged_sentence: 
        return None
    pos_tags = [tag for word, tag in tagged_sentence]
    chunk_pos_tags = self.chunk_tagger.tag(pos_tags)
    chunk_tags = [chunk_tag for (pos_tag, chunk_tag) in chunk_pos_tags]
    wpc_tags = [(word, pos_tag, chunk_tag) for ((word, pos_tag), chunk_tag)
                     in zip(tagged_sentence, chunk_tags)]
    return conlltags2tree(wpc_tags)


In [36]:
# We call our new class and pass it the training data from the chunked treebank
ntc = NGramTagChunker(train_data)
print(ntc.evaluate(test_data))
# Now we get really good results on the data set
# Why?

ChunkParse score:
    IOB Accuracy:  97.2%%
    Precision:     91.4%%
    Recall:        94.3%%
    F-Measure:     92.8%%


In [37]:
# Let's try to visualize the chunk for the more realistic sample sentence
tree = ntc.parse(tagged_sent)
print(tree)
tree.draw()


(S
  (NP the/DT brown/JJ fox/NN)
  is/VBZ
  (NP quick/JJ)
  and/CC
  (NP he/PRP)
  may/MD
  jump/VB
  over/IN
  (NP the/DT lazy/JJ dog/NN))


In [ ]:
![image3]()

In [38]:
# We now use our shallow parser on a larger 'Wall Street Journal' corpus
from nltk.corpus import conll2000
wsj_data = conll2000.chunked_sents()
train_wsj_data = wsj_data[:7500]
test_wsj_data = wsj_data[7500:]
print(train_wsj_data[10])

(S
  (NP He/PRP)
  (VP reckons/VBZ)
  (NP the/DT current/JJ account/NN deficit/NN)
  (VP will/MD narrow/VB)
  (PP to/TO)
  (NP only/RB #/# 1.8/CD billion/CD)
  (PP in/IN)
  (NP September/NNP)
  ./.)


In [39]:
# SAQ 1. How big is it?
len(wsj_data)

10948

In [40]:
# SAQ 2. How much test data did we have before, and how much now?
len(test_wsj_data)

3448

In [41]:
# We first train our model on the training corpus
tc = NGramTagChunker(train_wsj_data)
# And then we test it on the test data
print(tc.evaluate(test_wsj_data))

ChunkParse score:
    IOB Accuracy:  89.4%%
    Precision:     80.8%%
    Recall:        86.0%%
    F-Measure:     83.3%%
